# Import and Check Data

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Finance_Ecommerce.csv to Finance_Ecommerce.csv


In [3]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv("Finance_Ecommerce.csv")
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6060 entries, 0 to 6059
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    6057 non-null   object 
 1   Date             6060 non-null   object 
 2   AccountID        6059 non-null   object 
 3   AccountName      6059 non-null   object 
 4   TransactionType  6058 non-null   object 
 5   Amount           6059 non-null   object 
 6   Currency         6055 non-null   object 
 7   ExchangeRate     6058 non-null   float64
 8   Balance          5988 non-null   object 
 9   Merchant         6058 non-null   object 
 10  MerchantPhone    4647 non-null   float64
 11  MerchantEmail    4222 non-null   object 
 12  Category         6058 non-null   object 
 13  Subcategory      6055 non-null   object 
 14  Country          6058 non-null   object 
 15  City             6058 non-null   object 
 16  PostalCode       5742 non-null   float64
 17  CardNumber    

In [5]:
df.isnull().sum()

,0
TransactionID,3
Date,0
AccountID,1
AccountName,1
TransactionType,2
Amount,1
Currency,5
ExchangeRate,2
Balance,72
Merchant,2


In [6]:
df.duplicated().sum()

np.int64(1)

In [7]:
df.describe()

,ExchangeRate,MerchantPhone,PostalCode,Phone
count,6058.000000,4.647000e+03,5742.000000,5.732000e+03
mean,46.319508,3.094016e+09,534220.496517,2.977499e+09
std,40.254595,4.852691e+09,274401.478310,4.946857e+09
min,1.000000,-9.981343e+09,102.000000,-9.985638e+09
25%,1.000000,7.206737e+08,299628.500000,6.050150e+08
50%,72.091800,3.827888e+09,538097.500000,3.779223e+09
75%,80.274425,6.838675e+09,775529.500000,6.829491e+09
max,109.968400,9.999725e+09,999962.000000,9.999725e+09


# Data Cleaning

**Fix Inconsistent (Lower/Upper case)**

In [8]:
df['TransactionType'] = df['TransactionType'].str.upper().str.strip()
df['Currency'] = df['Currency'].str.upper().str.strip()
df['Category'] = df['Category'].str.title().str.strip()
df['Subcategory'] = df['Subcategory'].str.title().str.strip()
df['IsFraud'] = df['IsFraud'].str.upper().str.strip()

In [9]:
df['AccountName'] = df['AccountName'].str.strip().str.title()

**Fix Inconsistent (Remove $ , - from Amount)**

In [10]:
df["Amount"] = df["Amount"].astype(str)
df["Amount"] = df["Amount"].str.replace(r"[^\d\.\-]", "", regex=True)
df["Amount"] = pd.to_numeric(df["Amount"], errors="coerce")
df["Amount"] = df["Amount"].round(4)

**Replace INRR with INR and YESS with YES**

In [11]:
df['Currency'] = df['Currency'].replace('INRR', 'INR')
df['IsFraud'] = df['IsFraud'].replace('YESS', 'YES')

**Handle Missing and Duplicate Data (TransationID)**

In [12]:
df[df['TransactionID'].isnull()]

,TransactionID,Date,AccountID,AccountName,TransactionType,Amount,Currency,ExchangeRate,Balance,Merchant,...,Subcategory,Country,City,PostalCode,CardNumber,Email,Phone,IsFraud,Notes,CustomerSince
2997,NaN,7/15/2024,AC7251,Ritu Sharma,NaN,6.59,USD,NaN,-1078118.8,AutoZone,...,NaN,USA,Pune,396388.0,XXXX-XXXX-6226,NaN,8.783292e+09,NO,cashback credited,8/22/2024
4125,NaN,7/15/2025,AC8639,Karan Joshi,CREDIT,12.79,NaN,1.0,NaN,NaN,...,NaN,Canada,Kolkata,566942.0,NaN,karan.joshi@gmail.com,7.416538e+09,NO,cashback credited,2/11/2023
4234,NaN,1/22/2025,AC8560,Rohit Joshi,DEBIT,NaN,NaN,1.0,NaN,Zomato,...,Fastfood,Australia,Gurugram,959160.0,XXXX-XXXX-1733,rohit.joshi@yahoo.com,3.699892e+09,NO,suspected fraud,Jul-24


In [13]:
df = df.dropna(subset=['TransactionID'])

In [14]:
duplicates = df[df['TransactionID'].duplicated(keep=False)]
duplicates.sort_values('TransactionID')

,TransactionID,Date,AccountID,AccountName,TransactionType,Amount,Currency,ExchangeRate,Balance,Merchant,...,Subcategory,Country,City,PostalCode,CardNumber,Email,Phone,IsFraud,Notes,CustomerSince
2924,TX100234,7/1/2024,AC9399,Vikram Sharma,DEBIT,433.86,AED,18.4118,"?-182,672.26",Reliance,...,Concert,UAE,Gurugram,892667.0,4.4912E+15,vikram.sharma@gmail.com,NaN,NO,repeat purchase,2018
5193,TX100234,6/2/2025,AC9399,Vikram Sharma,DEBIT,433.86,AED,18.4118,"?-182,672.26",Reliance,...,Concert,UAE,Gurugram,892667.0,4.4912E+15,vikram.sharma@gmail.com,NaN,NO,repeat purchase,2018
1940,TX100312,5/19/2025,AC2513,Ritu Singh,CREDIT,89.17,USD,76.9680,"?-175,307.66",ElectroWorld,...,Cafe,Australia,Delhi,324757.0,XXXX-XXXX-XXXX-9589,ritu.singh@hotmail.com,5.404707e+09,NO,NaN,3/23/2024
6024,TX100312,8/3/2024,AC2513,Ritu Singh,CREDIT,89.17,USD,76.9680,"?-175,307.66",ElectroWorld,...,Cafe,Australia,Delhi,324757.0,XXXX-XXXX-XXXX-9589,ritu.singh@hotmail.com,5.404707e+09,NO,NaN,3/23/2024
1935,TX100330,7/31/2025,AC2233,Isha Singh,DEBIT,32.63,INR,1.0000,"?-176,002.59",LocalMart,...,Concert,Australia,Gurugram,111522.0,XXXX-XXXX-2215,isha.singh@gmail.com,-1.162887e+09,NO,repeat purchase,21/10/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5262,TX105584,4/11/2025,AC3693,Amit Das,CREDIT,134.43,INR,1.0000,-2868672.38,LocalMart,...,Electricity,UAE,Jaipur,472833.0,XXXX-XXXX-2153,amit.das@gmail.com,-1.277396e+09,NO,manual adjustment,2019
4368,TX105743,3/27/2024,AC1362,Arjun Gupta,CREDIT,5.70,USD,79.7449,-2901950.27,Myntra,...,Accessory,Australia,Noida,957228.0,XXXX-XXXX-7774,arjun.gupta@yahoo.com,4.178142e+09,NO,monthly subscription,3/7/2019
1044,TX105743,12/14/2024,AC1362,Arjun Gupta,CREDIT,5.70,USD,79.7449,-2901950.27,Myntra,...,Accessory,Australia,Noida,957228.0,XXXX-XXXX-7774,arjun.gupta@yahoo.com,4.178142e+09,NO,monthly subscription,3/7/2019
5450,TX105773,9/23/2024,AC8737,Karan Patel,DEBIT,12.14,USD,80.6438,-2902597.24,Zomato,...,Internet,UAE,Jaipur,347835.0,NaN,karan.patel@yahoo.com,9.724359e+09,NO,suspected fraud,10/3/2022


**Handle Missing Value (AccountID and AccountName)**

In [15]:
df[df['AccountID'].isnull()]
df[df['AccountName'].isnull()]

,TransactionID,Date,AccountID,AccountName,TransactionType,Amount,Currency,ExchangeRate,Balance,Merchant,...,Subcategory,Country,City,PostalCode,CardNumber,Email,Phone,IsFraud,Notes,CustomerSince
5823,TX101091,8/11/2024,NaN,NaN,DEBIT,31.67,INR,1.0,-554526.93,Flipkart,...,NaN,Canada,Bengaluru,273272.0,4.83654E+15,NaN,NaN,NO,NaN,unknown


In [16]:
df = df.dropna(subset=['AccountID'])

**Handle Missing Value (TransactionType)**

In [17]:
df['TransactionType'] = df['TransactionType'].fillna('NA')

**Handle Missing Value (Currency)**

In [18]:
df.groupby('Currency')['ExchangeRate'].describe()

,count,mean,std,min,25%,50%,75%,max
Currency,,,,,,,,
AED,979.0,19.952353,1.130182,18.0010,19.008950,19.90700,20.905650,21.9980
GBP,1024.0,99.874534,5.770835,90.0011,94.890825,99.66265,104.912975,109.9684
INR,2029.0,1.000000,0.000000,1.0000,1.000000,1.00000,1.000000,1.0000
USD,2020.0,77.521427,3.118366,72.0079,74.828750,77.53735,80.172425,82.9964


In [19]:
def infer_currency(rate):
    if pd.isna(rate):
        return None
    elif 72.0079 <= rate <= 82.9964:
        return "USD"
    elif 90.0011 <= rate <= 109.9684:
        return "GBP"
    elif 18.0010 <= rate <= 21.9980:
        return "AED"
    elif rate == 1:
        return "INR"
    else:
        return "NA"

mask = df["Currency"].isna()
df.loc[mask, "Currency"] = df.loc[mask, "ExchangeRate"].apply(infer_currency)



**Handle Missing Value (ExchangeRate)**

In [20]:
df['ExchangeRate'] = df.groupby('Currency')['ExchangeRate'].transform(
    lambda x: x.fillna(x.median())
)

**Fix Inconsistent & Handle Missing Value (Balance)**

In [21]:
df['Balance'] = (
    df['Balance']
    .astype(str)
    .str.replace('?', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.strip()
)

In [22]:
df['Balance'] = pd.to_numeric(df['Balance'], errors='coerce')
df['Balance'] = df['Balance'].round(4)

In [23]:
df['Balance'] = df['Balance'].fillna('NA')

In [24]:
df['Balance'] = pd.to_numeric(df['Balance'], errors='coerce')

**Handle Missing Value (Merchant, Subcategory, Country, City, IsFraud)**


In [25]:
df['Merchant'] = df['Merchant'].fillna('NA')
df['Subcategory'] = df['Subcategory'].fillna('NA')
df['Country'] = df['Country'].fillna('NA')
df['City'] = df['City'].fillna('NA')
df['IsFraud'] = df['IsFraud'].fillna('NA')

**Delete Column (MerchantPhone, MerchantEmail, PostalCode, CardNumber, Email, Phone)**

In [26]:
df = df.drop(columns=['MerchantPhone','MerchantEmail','PostalCode','Phone','Email','CardNumber'])

**Fix Inconsistent & Handle Missing Data (CustomerSince)**

In [27]:
import re
import pandas as pd

def extract_year(x):
    if pd.isna(x):
        return 'NA'

    x = str(x).strip().lower()

    if x in ['unknown', '', 'nan']:
        return 'NA'

    # Year only (e.g. 2025)
    if x.isdigit() and len(x) == 4:
        return x

    # Aug-15 → 2015
    if '-' in x and len(x.split('-')[-1]) == 2:
        return '20' + x.split('-')[-1]

    # Any format with 4-digit year (10/12/2017, 2017-05-02, etc.)
    match = re.search(r'\d{4}', x)
    if match:
        return match.group()

    return 'NA'


df['CustomerSince'] = df['CustomerSince'].apply(extract_year)

In [28]:
df['CustomerSince'] = pd.to_numeric(df['CustomerSince'], errors='coerce')

In [29]:
df['CustomerSince'] = df['CustomerSince'].fillna('NA')

**Handle Missing Data (Notes)**

In [30]:
df['Notes'] = df['Notes'].fillna('NA')

**Remove Duplicates Rows**

In [31]:
df = df.drop_duplicates()

# Download Data

In [32]:
df.to_csv("Finance_Cleaned.csv", index=False)